In [2]:
!pip install treelib

  Running setup.py bdist_wheel for treelib ... done
  Stored in directory: /home/ec2-user/.cache/pip/wheels/e4/1f/b0/8730974894530362affb8a242aee8f3b42d55ec822c9e2a520
Successfully built treelib
You are using pip version 10.0.1, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [1]:
import pandas as pd
from treelib import Tree, Node
 
org_df = pd.read_csv("test.csv", header=0, usecols=["name", "emp_id", "manager_id"],
                                 names=["name", "emp_id", "manager_id"], 
                                 dtype=object)
org_df = org_df[pd.notnull(org_df['emp_id'])]
org_df.loc[(org_df['manager_id'].isin(org_df["emp_id"]) | org_df['manager_id'].isna() ), 'manager_valid'] = True 
org_df["manager_valid"].fillna(False, inplace=True)
org_df["manager_id"].fillna('', inplace=True)
org_df

,name,emp_id,manager_id,manager_valid
0,Alan,100,150,True
1,Martin,220,100,True
2,Jamie,150,,True
3,Alex,275,100,True
4,Steve,400,150,True
5,David,190,400,True
6,John,460,500,False
7,Jason,660,190,True


In [2]:
my_org_tree = Tree()
my_org_tree.create_node("Root", "1")
my_org_tree.create_node("Unknown Manager", "1.1", parent = "1", data = {"name": "Unknown Manager", "id": "1.1"})

for index, row in org_df.iterrows():
    my_org_tree.create_node(row['name'], str(row['emp_id']), parent = "1", data = {"name":str(row['emp_id']), "id": str(row['emp_id'])})

for index, row in org_df.iterrows():
    if row['manager_id'] and row['manager_valid'] :
        parent = str(row['manager_id'])
    elif not row['manager_valid']:
        parent = "1.1"
    else:
        parent = "1"  
    my_org_tree.move_node(str(row['emp_id'] ), parent)

my_org_tree.show()

Root
├── Jamie
│   ├── Alan
│   │   ├── Alex
│   │   └── Martin
│   └── Steve
│       └── David
│           └── Jason
└── Unknown Manager
    └── John



In [3]:
org_dict = my_org_tree.to_dict(with_data=True)
org_obj = org_dict["Root"]['children']

In [10]:
def get_depth(id):
    node = my_org_tree.get_node(str(id))
    return my_org_tree.depth(node)


def create_emp_list(d, emp_list):
  if isinstance(d, (list)):
    for l in d:
        create_emp_list(l,emp_list)
  else:
    for k, v in d.items():
        if v.get("data"):
             temp_dict = {} 
             for n in range(1,my_org_tree.depth()+1):
                current_node = get_depth(v["data"]["id"])
                if n != current_node:
                    temp_dict.update({n:""})
                else:
                    temp_dict.update({current_node: k})
             emp_list.append(temp_dict)
        if v.get("children"):
            create_emp_list(v['children'],emp_list)
  return emp_list        

In [9]:
#org_dict
employees = create_emp_list(org_obj,[])
employees

[{1: 'Jamie', 2: '', 3: '', 4: ''},
 {1: '', 2: 'Alan', 3: '', 4: ''},
 {1: '', 2: '', 3: 'Alex', 4: ''},
 {1: '', 2: '', 3: 'Martin', 4: ''},
 {1: '', 2: 'Steve', 3: '', 4: ''},
 {1: '', 2: '', 3: 'David', 4: ''},
 {1: '', 2: '', 3: '', 4: 'Jason'},
 {1: 'Unknown Manager', 2: '', 3: '', 4: ''},
 {1: '', 2: 'John', 3: '', 4: ''}]

In [8]:
df = pd.DataFrame(employees) 
df

,1,2,3,4
0,Jamie,,,
1,,Alan,,
2,,,Alex,
3,,,Martin,
4,,Steve,,
5,,,David,
6,,,,Jason
7,Unknown Manager,,,
8,,John,,


In [491]:
df.to_csv('output.csv', header=False, index=False)